# 📈 Multi-Agent Stock Investment System

## 📝 Problem Statement  
The objective of this project is to build an intelligent **multi-agent financial assistant** capable of:

- 🔍 **Analyzing** stock market trends and signals  
- 📊 **Identifying** potential investment opportunities   
- 🧠 Utilizing **tools**, **memory**, and **planning** modules to enable autonomous decision-making  


The multi-agent stock investment system implements a BUY / HOLD / SELL behavior.




## 🔧 Base Agent

`BaseAgent` is a lightweight parent class that provides:
- A unique agent name
- Simple logging via `log(...)`

All other agents inherit from this for consistent structure and observability.

In [1]:
import yfinance as yf
import random
from typing import List, Dict


class BaseAgent:
    def __init__(self, name: str):
        self.name = name
        self.logs = []

    def log(self, message: str):
        entry = f"[{self.name}] {message}"
        self.logs.append(entry)
        print(entry)

## 📊 MarketAgent

**Responsibility**
- Analyze recent price history
- Compute 5-day percentage change
- Map it to one of:
  - `strong_bullish`, `weak_bullish`, `neutral`, `weak_bearish`, `strong_bearish`

This is the system's primary trend signal.

In [2]:
class MarketAgent(BaseAgent):
    def __init__(self):
        super().__init__("MarketAgent")

    def analyze(self, prices: List[float]) -> str:
        start = prices[0]
        end = prices[-1]

        pct_change = ((end - start) / start) * 100
        self.log(f"5-day % change: {pct_change:.2f}%")

        if pct_change > 3:
            return "strong_bullish"
        elif pct_change > 0.5:
            return "weak_bullish"
        elif pct_change > -1:
            return "neutral"
        elif pct_change > -3:
            return "weak_bearish"
        else:
            return "strong_bearish"


## 📰 NewsAgent

**Responsibility**
- Inspect synthetic headlines
- Classify sentiment as:
  - `positive` (if any headline contains "gain" or "growth")
  - otherwise `neutral`

This is a very simple stand-in for a real sentiment model.

In [3]:
class NewsAgent(BaseAgent):
    def __init__(self):
        super().__init__("NewsAgent")

    def analyze(self, headlines: List[str]) -> str:
        positive = any("gain" in h.lower() or "growth" in h.lower() for h in headlines)
        sentiment = "positive" if positive else "neutral"
        self.log(f"News sentiment: {sentiment}")
        return sentiment


## ⚠️ RiskAgent

**Responsibility**
- Compute volatility as `(max - min) / average`
- Map it to risk levels:
  - `low`, `medium`, `high`

This provides a coarse risk signal derived purely from price variability.

In [4]:
class RiskAgent(BaseAgent):
    def __init__(self):
        super().__init__("RiskAgent")

    def assess(self, prices: List[float]) -> str:
        volatility = max(prices) - min(prices)
        avg_price = sum(prices) / len(prices)
        vol_ratio = volatility / avg_price
        self.log(f"Volatility ratio: {vol_ratio:.3f}")

        if vol_ratio < 0.02:
            return "low"
        elif vol_ratio < 0.10:
            return "medium"
        elif vol_ratio < 0.25:
            return "high"
        else:
            return "extreme"

## ✅ DecisionAgent

**Responsibility**
- Combine:
  - Market trend (`MarketAgent`)
  - News sentiment (`NewsAgent`)
  - Risk level (`RiskAgent`)
- Produce final action: `BUY`, `HOLD`, or `SELL`

**Design choices**
- Softer negative weights for bearish markets
- Stronger boosts for positive news and low risk

In [5]:
class DecisionAgent(BaseAgent):
    def __init__(self):
        super().__init__("DecisionAgent")

    def decide(self, market: str, news: str, risk: str) -> str:
        score = 0

        # Market contribution (softened strong_bearish)
        market_weights = {
            "strong_bullish": 2,
            "weak_bullish": 1,
            "neutral": 0,
            "weak_bearish": -1,
            "strong_bearish": -1,  # softer than -2
        }
        score += market_weights.get(market, 0)

        # News contribution
        if news == "positive":
            score += 2
        # neutral -> 0

        # Risk contribution
        if risk == "low":
            score += 2
        elif risk == "medium":
            score += 0
        elif risk == "high":
            score -= 1
        elif risk == "extreme":
            score -= 2


        self.log(f"Final score: {score}")

        # Thresholds:
        # - BUY: clearly positive
        # - SELL: clearly negative
        # - HOLD: everything in between
        if score >= 2:
            return "BUY"
        elif score <= -3:
            return "SELL"
        else:
            return "HOLD"

## 🧠 MemoryAgent

**Responsibility**
- Store past:
  - prices
  - headlines
  - decisions

This simulates a simple long-term memory buffer for future analysis or learning.

In [6]:
class MemoryAgent(BaseAgent):
    def __init__(self):
        super().__init__("MemoryAgent")
        self.history = []

    def store_decision(self, prices, headlines, decision):
        self.history.append(
            {
                "prices": prices,
                "headlines": headlines,
                "decision": decision,
            }
        )
        self.log("Decision stored in memory")



## 🧮 EvaluationAgent

**Responsibility**
- Assign a simple quality score to the final decision
- Here, we simulate an evaluation by sampling a random score in [0.7, 0.95]

In a real system, this could be an LLM-as-a-Judge or a backtesting module.

In [7]:
class EvaluationAgent(BaseAgent):
    def __init__(self):
        super().__init__("EvaluationAgent")

    def evaluate(self, decision: str) -> Dict:
        score = round(random.uniform(0.7, 0.95), 2)
        self.log(f"QualityScore={score}")
        return {"score": score}

## 🏗️ InvestmentSystem

**Responsibility**
- Orchestrate all agents:
  - Run market, news, and risk analysis
  - Call `DecisionAgent` to get BUY/HOLD/SELL
  - Store decision in memory
  - Evaluate decision quality
- Provide a `run_for_stock` helper that:
  - Fetches recent prices with `yfinance`
  - Uses synthetic headlines
  - Prints a human-readable summary

In [8]:
class InvestmentSystem:
    def __init__(self):
        self.market_agent = MarketAgent()
        self.news_agent = NewsAgent()
        self.risk_agent = RiskAgent()
        self.decision_agent = DecisionAgent()
        self.memory_agent = MemoryAgent()
        self.evaluation_agent = EvaluationAgent()

    def run(self, prices: List[float], headlines: List[str]) -> Dict:
        market_signal = self.market_agent.analyze(prices)
        news_signal = self.news_agent.analyze(headlines)
        risk_profile = self.risk_agent.assess(prices)

        decision = self.decision_agent.decide(
            market_signal,
            news_signal,
            risk_profile,
        )

        self.memory_agent.store_decision(prices, headlines, decision)
        eval_result = self.evaluation_agent.evaluate(decision)

        return {
            "decision": decision,
            "evaluation": eval_result,
            "market": market_signal,
            "news": news_signal,
            "risk": risk_profile,
        }

    def run_for_stock(self, stock: str) -> Dict:
        prices = get_last_5_day_prices(stock)
        headlines = get_recent_headlines(stock)

        print(f"\n📈 {stock} prices (last 5 days): {prices}")
        print(f"📰 {stock} headlines: {headlines}")

        result = self.run(prices, headlines)
        print("\nFINAL DECISION:", result["decision"])
        print("DETAIL:", result)
        return result

## 🔗 Data Fetch Helpers

These helper functions:
- `get_last_5_day_prices`:
  - Pulls ~1 week of prices via `yfinance`
  - Ensures at least 5 trading days
- `get_recent_headlines`:
  - Returns synthetic headlines for demonstration

In [9]:
def get_last_5_day_prices(stock: str) -> List[float]:
    ticker = yf.Ticker(stock)
    hist = ticker.history(period="7d")  # ensures 5 trading days
    prices = hist["Close"].dropna().tolist()

    if len(prices) < 5:
        raise ValueError("Not enough market data")

    return prices[-5:]


def get_recent_headlines(stock: str) -> List[str]:
    return [
        f"{stock} shows movement amid broader market conditions",
        f"Analysts discuss near-term outlook for {stock}",
        f"Investor sentiment around {stock} remains cautious",
    ]

## 🧪 Test Cases: BUY, HOLD, SELL

The following cells construct synthetic scenarios to explicitly test:
- A case that should produce BUY
- A case that should produce HOLD
- A case that should produce SELL

These use manual `prices` and `headlines` so you don't depend on live data.


### BUY Test Case

Conditions:
- Clear 5-day uptrend -> bullish
- Headline contains the word "growth" -> positive news
- Volatility moderate -> likely not "high" risk

In [10]:
system = InvestmentSystem()

# Prices: clear uptrend -> strong_bullish
prices_buy = [100, 101, 102, 103, 105]

# Headline contains "growth" -> positive news
headlines_buy = [
    "Company reports strong revenue growth this quarter",
    "Analysts see further upside",
]

result_buy = system.run(prices_buy, headlines_buy)
print("\n=== BUY TEST ===")
print(result_buy)

[MarketAgent] 5-day % change: 5.00%
[NewsAgent] News sentiment: positive
[RiskAgent] Volatility ratio: 0.049
[DecisionAgent] Final score: 4
[MemoryAgent] Decision stored in memory
[EvaluationAgent] QualityScore=0.72

=== BUY TEST ===
{'decision': 'BUY', 'evaluation': {'score': 0.72}, 'market': 'strong_bullish', 'news': 'positive', 'risk': 'medium'}


### HOLD Test Case

Conditions:
- Clear downtrend -> bearish
- Neutral headlines (no "gain" or "growth")
- Volatility elevated but not extreme

With the softened weights and thresholds, this should result in HOLD.

In [11]:
system = InvestmentSystem()

# Prices: clear downtrend -> strong_bearish
prices_hold = [105, 103, 102, 101, 99]

# No "gain" or "growth" -> neutral news
headlines_hold = [
    "Stock shows movement amid broader market conditions",
    "Analysts discuss near-term outlook for the stock",
    "Investor sentiment around the stock remains cautious",
]

result_hold = system.run(prices_hold, headlines_hold)
print("\n=== HOLD TEST ===")
print(result_hold)

[MarketAgent] 5-day % change: -5.71%
[NewsAgent] News sentiment: neutral
[RiskAgent] Volatility ratio: 0.059
[DecisionAgent] Final score: -1
[MemoryAgent] Decision stored in memory
[EvaluationAgent] QualityScore=0.78

=== HOLD TEST ===
{'decision': 'HOLD', 'evaluation': {'score': 0.78}, 'market': 'strong_bearish', 'news': 'neutral', 'risk': 'medium'}


### SELL Test Case

Conditions:
- Very strong 5-day downtrend -> strong_bearish
- Neutral / slightly negative headlines
- Volatility high -> risk may be "high"

This should push the score to <= -3 and result in SELL.

In [12]:
system = InvestmentSystem()

# Prices: very strong downtrend -> strong_bearish with high volatility
prices_sell = [120, 110, 100, 90, 80]

# Still neutral for our simple sentiment logic (no "gain" or "growth")
headlines_sell = [
    "Stock faces pressure amid broader market concerns",
    "Analysts highlight short-term headwinds",
]

result_sell = system.run(prices_sell, headlines_sell)
print("\n=== SELL TEST ===")
print(result_sell)

[MarketAgent] 5-day % change: -33.33%
[NewsAgent] News sentiment: neutral
[RiskAgent] Volatility ratio: 0.400
[DecisionAgent] Final score: -3
[MemoryAgent] Decision stored in memory
[EvaluationAgent] QualityScore=0.83

=== SELL TEST ===
{'decision': 'SELL', 'evaluation': {'score': 0.83}, 'market': 'strong_bearish', 'news': 'neutral', 'risk': 'extreme'}


In [13]:
# Real-world test with a live ticker
system = InvestmentSystem()

stock = "AAPL"
prices_real = get_last_5_day_prices(stock)

headlines_real = [
    f"{stock} reports strong earnings growth and market share gains",
    f"Analysts see further upside potential for {stock}",
]

print(f"\nUsing real prices for {stock}: {prices_real}")
print(f"Headlines: {headlines_real}")

result_real = system.run(prices_real, headlines_real)
print("\n=== REAL-WORLD TEST ===")
print(result_real)


Using real prices for AAPL: [259.4800109863281, 270.010009765625, 269.4800109863281, 276.489990234375, 276.17999267578125]
Headlines: ['AAPL reports strong earnings growth and market share gains', 'Analysts see further upside potential for AAPL']
[MarketAgent] 5-day % change: 6.44%
[NewsAgent] News sentiment: positive
[RiskAgent] Volatility ratio: 0.063
[DecisionAgent] Final score: 4
[MemoryAgent] Decision stored in memory
[EvaluationAgent] QualityScore=0.72

=== REAL-WORLD TEST ===
{'decision': 'BUY', 'evaluation': {'score': 0.72}, 'market': 'strong_bullish', 'news': 'positive', 'risk': 'medium'}
